In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.special import expit
%matplotlib inline

In [2]:
class neuralNetwork:
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        self.lr = learningrate
        # Initial weights
        self.wih = np.random.normal(0, pow(self.hnodes, -0.5), (self.hnodes, self.inodes))
        self.who = np.random.normal(0, pow(self.onodes, -0.5), (self.onodes, self.hnodes))
        
    def activation_function(self, x):
        """Sigmoid activation function. """
        return expit(x)
    
    def train(self, inputs_list, targets_list):
        """
        Method for training neural network.
        Arguments are python lists.
        """
        inputs = np.array(inputs_list, ndmin=2).T
        targets = np.array(targets_list, ndmin=2).T
        
        hidden_inputs = np.dot(self.wih, inputs)
        hidden_outputs = self.activation_function(hidden_inputs)
        
        final_inputs = np.dot(self.who, hidden_outputs)
        final_outputs = self.activation_function(final_inputs)
        
        output_errors = targets - final_outputs
        hidden_errors = np.dot(self.who.T, output_errors)
        
        self.who += self.lr * np.dot((output_errors * final_outputs * (1 - final_outputs)), hidden_outputs.T)
        self.wih += self.lr * np.dot((hidden_errors * hidden_outputs * (1 - hidden_outputs)), inputs.T)
    
    def query(self, input_list):
        inputs = np.array(input_list, ndmin=2).T
        hidden_inputs = np.dot(self.wih, inputs)
        hidden_outputs = self.activation_function(hidden_inputs)
        final_inputs = np.dot(self.who, hidden_outputs)
        final_outputs = self.activation_function(final_inputs)
        return final_outputs

In [3]:
input_nodes, hidden_nodes, output_nodes = 784, 100, 10
learning_rate = 0.3
n = neuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)

In [4]:
training_df = pd.read_csv('mnist_dataset/mnist_train_100.csv', header=None)

In [5]:
inputs = (training_df.loc[:, 1:]/255.*0.99) + 0.01

In [6]:
targets = pd.DataFrame(columns=range(10), index=range(len(training_df)))
targets.fillna(0.01, inplace=True)
for ind, targ in enumerate(training_df[0]):
    targets.loc[ind, targ] = 0.99

In [7]:
for ((ind_i, row_i), (ind_t, row_t)) in zip(inputs.iterrows(), targets.iterrows()):
    n.train(row_i.tolist(), row_t.tolist())

In [8]:
testing_df = pd.read_csv('mnist_dataset/mnist_test_10.csv', header=None)

In [10]:
correct_labels = testing_df.loc[:, 0].tolist()
inputs = (testing_df.loc[:, 1:]/255.*0.99) + 0.01
outputs = [n.query(inp).argmax() for (_, inp) in inputs.iterrows()]
scorecard = [1 if x == y else 0 for (x, y) in zip(outputs, correct_labels)]

In [13]:
print(scorecard)

[1, 1, 1, 1, 1, 1, 1, 0, 0, 0]
